In [130]:
from adam.casadi import KinDynComputations
import casadi as cs
from math import sqrt
import numpy as np

In [131]:
model_path = "/Users/tommasoandina/Desktop/doosan-robot2-master/dsr_description2/urdf/h2515.blue.urdf" 
joints_name_list = 'joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6' 
root_link = 'base'

kinDyn = KinDynComputations(model_path, joints_name_list, root_link)
num_dof = kinDyn.NDoF

In [132]:
H = cs.SX.sym('H', 4, 4)
# The joint values
s = cs.SX.sym('s', num_dof)
# The base velocity
v_b = cs.SX.sym('v_b', 6)
# The joints velocity
s_dot = cs.SX.sym('s_dot', num_dof)
# The base acceleration
v_b_dot = cs.SX.sym('v_b_dot', 6)
# The joints acceleration
s_ddot = cs.SX.sym('s_ddot', num_dof)

# initialize
mass_matrix_fun = kinDyn.mass_matrix_fun()
coriolis_term_fun = kinDyn.coriolis_term_fun()
gravity_term_fun =  kinDyn.gravity_term_fun()
bias_force_fun = kinDyn.bias_force_fun()
Jacobian_fun = kinDyn.jacobian_fun("link6")


In [133]:

#print('Mass matrix:\n', cs.DM(M(H, s)))
#print('Mass matrix:\n', M(H, s))


In [134]:
h = kinDyn.bias_force_fun()
#print('Bias force term:\n', (h(H, s, v_b, s_dot)))

In [135]:
#Controll law (tau)

class Controller:
    def __init__(self, kp, kd, dt, q_des):
        self.q_previous = 0.0
        self.kp = kp
        self.kd = kd
        self.dt = dt
        self.q_des = q_des
        self.first_iter = True

    def control(self, q, dq):
        if self.first_iter:
            self.q_previous = q
            self.first_iter = False

        self.q_previous = q
        return self.kp * (self.q_des - q) - self.kd * dq

In [136]:
class Simulator:
    def __init__(self, q, dt, dq, ddq):
        self.q = q
        self.dt = dt
        self.dq = dq
        self.ddq = ddq

    def simulate_q(self, tau, h):
        dq = self.simulate_dq(tau, h)
        self.q += self.dt * dq
        return self.q
    
    def simulate_dq(self, tau, h):
        self.ddq = cs.inv(M) @ (tau - h)
        self.dq += self.dt * self.ddq
        return self.dq
    
    def simulate_ddq(self, M, tau, h):
        self.ddq = cs.inv(M) @ (tau - h)
        return self.ddq


In [137]:
q_0 = cs.SX.sym('q_0', num_dof)
kp = 0.1 
kd = sqrt(kp)
dt = 1.0 / 16.0 * 1e-3
total_time = 2.0 * 1e-3
q_des = cs.SX.sym('q_des', num_dof)
position = [200, 50, 100, 250, 300, -50]
q_des = cs.vertcat(*position)

dq = cs.SX.sym('dq', num_dof)
ddq = cs.SX.sym('ddq', num_dof)

N = int(total_time / dt)

ctrl = Controller(kp, kd, dt, q_des)
simu = Simulator(q_0, dt, dq, ddq)


In [138]:
M = mass_matrix_fun(H, s)  
M = M[:6, :6]

h = bias_force_fun(H, s, v_b, s_dot)  
h = h[:6]
print(h[:6])

for i in range(N):
    tau = ctrl.control(simu.q, simu.dq)
    simu.simulate_q(tau, h)
    simu.simulate_dq(tau, h)
    simu.simulate_ddq(M, tau, h)

@1=3.722, @2=-9.80665, @3=((((((H_2*v_b_4)-(H_1*v_b_5))*v_b_0)+(((H_0*v_b_5)-(H_2*v_b_3))*v_b_1))+(((H_1*v_b_3)-(H_0*v_b_4))*v_b_2))-(@2*H_2)), @4=(((H_4*v_b_3)+(H_5*v_b_4))+(H_6*v_b_5)), @5=(((H_8*v_b_0)+(H_9*v_b_1))+(H_10*v_b_2)), @6=(((H_8*v_b_3)+(H_9*v_b_4))+(H_10*v_b_5)), @7=(((H_4*v_b_0)+(H_5*v_b_1))+(H_6*v_b_2)), @8=-0.00022332, @9=-0.188791, @10=(((H_0*v_b_3)+(H_1*v_b_4))+(H_2*v_b_5)), @11=4.4664e-05, @12=-4.4664e-05, @13=cos(s_0), @14=7.446, @15=sin(s_0), @16=((((((H_6*v_b_4)-(H_5*v_b_5))*v_b_0)+(((H_4*v_b_5)-(H_6*v_b_3))*v_b_1))+(((H_5*v_b_3)-(H_4*v_b_4))*v_b_2))-(@2*H_6)), @17=(((H_0*v_b_0)+(H_1*v_b_1))+(H_2*v_b_2)), @18=-0.3443, @19=(@18*@13), @20=(@18*@15), @21=((((@13*@7)-(@15*@17))+(@19*@10))+(@20*@4)), @22=(((@13*@3)+(@15*@16))+(@21*s_dot_0)), @23=1.06721, @24=((@13*@10)+(@15*@4)), @25=(@24*s_dot_0), @26=((@13*@4)-(@15*@10)), @27=(@6+s_dot_0), @28=0.329716, @29=-1.06721, @30=-0.00052122, @31=0.00052122, @32=4.14828e-08, @33=cos(s_1), @34=1, @35=sin(s_1), @36=((@32*@33)+

In [139]:
error_q = q_des - simu.q
norm_inf_error = cs.norm_inf(error_q)
print(norm_inf_error , 'norm_inf_error')

@1=200, @2=6.25e-05, @3=3.722, @4=cos(s_0), @5=7.446, @6=4.14828e-08, @7=cos(s_1), @8=1, @9=sin(s_1), @10=((@6*@7)+(@8*@9)), @11=10.569, @12=-0.000203673, @13=cos(s_2), @14=-1, @15=sin(s_2), @16=((@12*@13)+(@14*@15)), @17=3.614, @18=cos(s_3), @19=3.785, @20=cos(s_4), @21=2.598, @22=1.128, @23=cos(s_5), @24=(@22*@23), @25=sin(s_5), @26=(@22*@25), @27=(@21+((@24*@23)+(@26*@25))), @28=sin(s_4), @29=(@12*@25), @30=(@22*@29), @31=(@12*@23), @32=(@22*@31), @33=((@30*@23)-(@32*@25)), @34=((@20*@27)-(@28*@33)), @35=((@24*@29)-(@26*@31)), @36=(@21+(((@30*@29)+(@32*@31))+1.128)), @37=((@20*@35)-(@28*@36)), @38=(@19+((@34*@20)-(@37*@28))), @39=sin(s_3), @40=(@12*@28), @41=(@12*@20), @42=(@8*@25), @43=(@22*@42), @44=(@8*@23), @45=(@22*@44), @46=((@43*@23)-(@45*@25)), @47=(((@40*@27)+(@41*@33))+(@8*@46)), @48=0.000229743, @49=(((@43*@29)+(@45*@31))+@48), @50=(((@40*@35)+(@41*@36))+(@8*@49)), @51=((@47*@20)-(@50*@28)), @52=((@18*@38)-(@39*@51)), @53=((@24*@42)-(@26*@44)), @54=(((@30*@42)+(@32*@44))+